In [1]:
#Imports
from comet_ml import Experiment
experiment = Experiment(api_key="YkPEmantOag1R1VOJmXz11hmt", parse_args=False, project_name='SegNet')

COMET INFO: old comet version (1.0.44) detected. current: 1.0.52 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Failing to collect the installed os packages
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yikeqicn/segnet/961f319a85784a09943874b209a48ae6



In [2]:
from os.path import join, basename, dirname
import tensorflow as tf
import shutil
import os
import sys
import argparse

import cv2
import numpy as np
import os
import time

from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
import torch
import torchvision
import torchvision.transforms as transforms
from datasets import ArtPrint, ArtPrintNoIntsect,ArtPrintNoIntsectBinary,ArtPrintNoIntsectLBW_bpr_spr
from Model_Unet_github import *

home = os.environ['HOME']

In [3]:
#General Settings
parser = argparse.ArgumentParser()

# system basics
parser.add_argument("-name", default='segnet_unet_lbw_bpr_spr_100_epoches', type=str, help="name of the log") #debug model_intersect # segnet_no_intersect_1conv_64_channels_30epoch_unet_github
parser.add_argument("-gpu", default='2', type=str, help="gpu numbers")

parser.add_argument("-train", default=True, help="train the NN", action="store_true")

parser.add_argument("-transfer",default=False, help="test the NN", action="store_true")

parser.add_argument("-test",default=False, help="test the NN", action="store_true")

# image and logistic parameters 
parser.add_argument("-image_h", default=32, type=int, help='image height') #('image_h', "360", """ image height """) 32
parser.add_argument("-image_w", default=128, type=int, help='image width')#('image_w', "480", """ image width """)128
#parser.add_argument("-image_h", default=360, type=int, help='image height') 
#parser.add_argument("-image_w", default=480, type=int, help='image width')

parser.add_argument("-image_c", default=1, type=int, help='image channel')#('image_c', "3", """ image channel (RGB) """)
parser.add_argument("-num_class", default=2, type=int, help='total class number')

# training hyperparam
parser.add_argument("-batch_size", default=10, type=int, help='batch_size')
parser.add_argument("-lrInit", default=1e-3, type=int, help='initial lr')
parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time') # not sure
parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time') # not sure
parser.add_argument('-max_epoch',default=100, type=int,help='max epoch numbers')



# file paths
parser.add_argument('-ckpt_root', default="/root/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
parser.add_argument('-data_root', default="/root/datasets", type=str, help=" root to any data folder ")
parser.add_argument('-urlTranferFrom', default="", type=str, help=" archived model url ")


#args = parser.parse_args()
args = parser.parse_known_args()[0]

name = args.name

experiment.set_name(name)
experiment.log_parameters(vars(args))

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

ckptroot = args.ckpt_root
args.ckptpath = join(ckptroot, name)
if args.name=='debug': shutil.rmtree(args.ckptpath, ignore_errors=True)
os.makedirs(args.ckptpath, exist_ok=True)


In [4]:
def checkArgs():
    if args.test:
        print('The model is set to Testing')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid testing dir: %s"%FLAGS.test_dir)
    elif args.transfer:
        print('The model is set to transfer learn from ckpt')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Image dir: %s"%FLAGS.image_dir)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)
    else:
        print('The model is set to Training')
        print("Max training Iteration: %d"%args.max_epoch)
        print("Initial lr: %f"%args.lrInit)
        print("First Drop Steps: %i"%args.lrDrop1)
        print("Second Drop Steps: %i"%args.lrDrop2)
        print("Data root: %s"%args.data_root)
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)

    print("Batch Size: %d"%args.batch_size)

In [5]:
#main function
def main():    
    checkArgs()
    if args.train:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        #arprint=ArtPrint(args.data_root, transform=transform_train)
        arprint=ArtPrintNoIntsectLBW_bpr_spr(args.data_root, transform=transform_train)
        concat=arprint
        print(len(arprint))
        idxTrain = int( len(arprint)*0.9)
        trainset, testset = random_split(concat, [idxTrain, len(concat)-idxTrain])
        trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, drop_last=True,num_workers=4)
        testloader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        
        #weight gen
        pos_perc=sum(map(lambda x: np.sum(cv2.imread(x[1],cv2.IMREAD_GRAYSCALE)),trainset.dataset.samples))/(args.image_h*args.image_w*len(trainset.dataset.samples))
        neg_perc=1-pos_perc
        weight=np.array([pos_perc,neg_perc]) # just reverse
        print(weight)
        model=Model(args, experiment, loss_weight=weight, mustRestore=False)
        model.training(loader=trainloader, validateloader=testloader)
        
    else:
        pass # for now, leave it pass 
    

In [ ]:
main()

The model is set to Training
Max training Iteration: 100
Initial lr: 0.001000
First Drop Steps: 10
Second Drop Steps: 1000
Data root: /root/datasets
Check point file: /root/ckpt/segnet_unet_lbw_bpr_spr_100_epoches
Batch Size: 10
284246
[0.10690873 0.89309127]
Instructions for updating:
Colocations handled automatically by placer.
GGG
[None, 32, 128, 1]
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


0 conv1: (?, ?, ?, 32)
0 conv2: (?, ?, ?, 32)
1 conv1: (?, ?, ?, 64)
1 conv2: (?, ?, ?, 64)
2 conv1: (?, ?, ?, 128)
2 conv2: (?, ?, ?, 128)
1 h_deconv: (?, ?, ?, 64)
1 h_deconv_concat: (?, ?, ?, ?)
1 h_conv1_post_deconv: (?, ?, ?, 64)
1 h_conv2_post_deconv: (?, ?, ?, 64)
0 h_deconv: (?, ?, ?, 32)
0 h_deconv_concat: (?, ?, ?, ?)
0 h_conv1_post_deconv: (?, ?, ?, 32)
0 h_conv2_post_deconv: (?, ?, ?, 32)
0 outmap: (?, ?, ?, 2)
(?, ?, ?, 2)
loss: ()
INFO:tensorflow:Summary name loss/cross_entropy (raw) is illegal; using loss/cross_entropy__raw_ instead.


...

TRAIN: Batch: 0.3830818544288953 Loss: 0.022960342
accuracy = 0.948318
mean IU  = 0.817834
    class # 0 capture rate = 0.944559 
    class # 1 capture rate = 0.975828 
TRAIN: Batch: 0.3869908529434759 Loss: 0.024795223
accuracy = 0.939647
mean IU  = 0.805803
    class # 0 capture rate = 0.933266 
    class # 1 capture rate = 0.981927 
TRAIN: Batch: 0.39089985145805645 Loss: 0.019714449
accuracy = 0.950585
mean IU  = 0.804975
    class # 0 capture rate = 0.948170 
    class # 1 capture rate = 0.972128 
TRAIN: Batch: 0.394808849972637 Loss: 0.024252731
accuracy = 0.940977
mean IU  = 0.796141
    class # 0 capture rate = 0.937855 
    class # 1 capture rate = 0.964256 
TRAIN: Batch: 0.3987178484872176 Loss: 0.02779914
accuracy = 0.928854
mean IU  = 0.753437
    class # 0 capture rate = 0.924276 
    class # 1 capture rate = 0.968295 
TRAIN: Batch: 0.4026268470017981 Loss: 0.0272307
accuracy = 0.932262
mean IU  = 0.791713
    class # 0 capture rate = 0.925183 
    class # 1 capture rate =

TRAIN: Batch: 0.574622781643343 Loss: 0.030896967
accuracy = 0.926540
mean IU  = 0.776650
    class # 0 capture rate = 0.921216 
    class # 1 capture rate = 0.960784 
TRAIN: Batch: 0.5785317801579235 Loss: 0.030456627
accuracy = 0.925772
mean IU  = 0.772499
    class # 0 capture rate = 0.919303 
    class # 1 capture rate = 0.968902 
TRAIN: Batch: 0.5824407786725041 Loss: 0.016333183
accuracy = 0.957757
mean IU  = 0.816031
    class # 0 capture rate = 0.955161 
    class # 1 capture rate = 0.983827 
TRAIN: Batch: 0.5863497771870847 Loss: 0.019534588
accuracy = 0.954715
mean IU  = 0.824612
    class # 0 capture rate = 0.952125 
    class # 1 capture rate = 0.976067 
TRAIN: Batch: 0.5902587757016652 Loss: 0.02213318
accuracy = 0.946730
mean IU  = 0.790742
    class # 0 capture rate = 0.944129 
    class # 1 capture rate = 0.970750 
TRAIN: Batch: 0.5941677742162458 Loss: 0.02036503
accuracy = 0.946378
mean IU  = 0.796236
    class # 0 capture rate = 0.943025 
    class # 1 capture rate =

TRAIN: Batch: 0.7661637088577906 Loss: 0.018493993
accuracy = 0.956199
mean IU  = 0.822442
    class # 0 capture rate = 0.953760 
    class # 1 capture rate = 0.977826 
TRAIN: Batch: 0.7700727073723712 Loss: 0.019364972
accuracy = 0.953414
mean IU  = 0.817137
    class # 0 capture rate = 0.950899 
    class # 1 capture rate = 0.974977 
TRAIN: Batch: 0.7739817058869518 Loss: 0.023542333
accuracy = 0.941498
mean IU  = 0.793530
    class # 0 capture rate = 0.937590 
    class # 1 capture rate = 0.972138 
TRAIN: Batch: 0.7778907044015323 Loss: 0.02534462
accuracy = 0.937337
mean IU  = 0.793656
    class # 0 capture rate = 0.933989 
    class # 1 capture rate = 0.960590 
TRAIN: Batch: 0.7817997029161129 Loss: 0.018196713
accuracy = 0.957396
mean IU  = 0.824477
    class # 0 capture rate = 0.955166 
    class # 1 capture rate = 0.977495 
TRAIN: Batch: 0.7857087014306935 Loss: 0.020770762
accuracy = 0.949893
mean IU  = 0.803137
    class # 0 capture rate = 0.947936 
    class # 1 capture rate

TRAIN: Batch: 0.9577046360722383 Loss: 0.023240987
accuracy = 0.944174
mean IU  = 0.810061
    class # 0 capture rate = 0.939100 
    class # 1 capture rate = 0.980342 
TRAIN: Batch: 0.9616136345868188 Loss: 0.024361301
accuracy = 0.941720
mean IU  = 0.802488
    class # 0 capture rate = 0.937465 
    class # 1 capture rate = 0.972244 
TRAIN: Batch: 0.9655226331013994 Loss: 0.024679627
accuracy = 0.946591
mean IU  = 0.812136
    class # 0 capture rate = 0.945145 
    class # 1 capture rate = 0.957074 
TRAIN: Batch: 0.96943163161598 Loss: 0.024135867
accuracy = 0.938068
mean IU  = 0.776592
    class # 0 capture rate = 0.934493 
    class # 1 capture rate = 0.968445 
TRAIN: Batch: 0.9733406301305606 Loss: 0.022967963
accuracy = 0.946648
mean IU  = 0.809505
    class # 0 capture rate = 0.943259 
    class # 1 capture rate = 0.972444 
TRAIN: Batch: 0.9772496286451411 Loss: 0.023628717
accuracy = 0.946361
mean IU  = 0.794607
    class # 0 capture rate = 0.944379 
    class # 1 capture rate 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
